#### **Explanation:**
* Simulates a self-aware agent that logs not only what it says, but also why it made that choice.
* Stores introspective reasoning alongside the agent's responses for each user input.
* Uses ChromaDB to persist:
** User messages
** Agent replies
** Agent reasoning (e.g., "chose to answer informally due to casual tone")
* Helps with debugging, self-reflection, and training autonomous agents.
* Enables future agents to review past decision traces to improve behavior over time.
* Interactive console-based interface with:
** **show** - view all logs (input, output, introspection)
** **clear** delete stored history
** **logout** switch user
** **exit** quit the program

In [ ]:
!pip install --upgrade chromadb --quiet
!pip install sentence-transformers --quiet

In [ ]:
import chromadb
from sentence_transformers import SentenceTransformer
from chromadb.config import Settings
import time
from datetime import datetime

In [ ]:
# Initialize ChromaDB persistent client
PERSIST_DIR = "/content/chroma_goal_agent"
chroma_client = chromadb.PersistentClient(path=PERSIST_DIR)

# Create or load a collection
COLLECTION_NAME = "goals"
collection = chroma_client.get_or_create_collection(name=COLLECTION_NAME)

In [ ]:
# Store a goal with metadata
def store_goal(user_id, goal_text):
    goal_id = f"{user_id}_{int(time.time())}"
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    metadatas = [{
        "timestamp": timestamp,
        "category": "general",
        "user_id": user_id
    }]
    collection.add(
        documents=[goal_text],
        ids=[goal_id],
        metadatas=metadatas
    )
    print("Goal stored successfully.")

In [ ]:
# Recall similar goals using ChromaDB's similarity search
def recall_goals(query_text, top_k=3):
    results = collection.query(query_texts=[query_text], n_results=top_k)
    matches = results.get("documents", [[]])[0]
    ids = results.get("ids", [[]])[0]
    metadata_list = results.get("metadatas", [[]])[0]

    if matches:
        print(f"Top {len(matches)} similar goals:")
        for i, (goal, meta) in enumerate(zip(matches, metadata_list), 1):
            timestamp = meta.get("timestamp", "unknown")
            category = meta.get("category", "general")
            print(f"{i}. Goal: {goal}")
            print(f"   Time: {timestamp} | Category: {category}")
    else:
        print("No similar goals found.")

In [ ]:
# View all stored goals with metadata
def view_all_goals():
    all_data = collection.get()
    docs = all_data.get("documents", [])
    metas = all_data.get("metadatas", [])
    ids = all_data.get("ids", [])

    if not docs:
        print("No goals stored yet.")
        return

    for i, (goal, meta, doc_id) in enumerate(zip(docs, metas, ids), 1):
        timestamp = meta.get("timestamp", "unknown")
        category = meta.get("category", "general")
        print(f"{i}. ID: {doc_id}")
        print(f"   Goal: {goal}")
        print(f"   Time: {timestamp} | Category: {category}")

In [ ]:
# Clear all stored goals
def clear_all_goals():
    all_data = collection.get()
    ids_to_delete = all_data.get("ids", [])
    if ids_to_delete:
        collection.delete(ids=ids_to_delete)
        print("All goals cleared.")
    else:
        print("No goals to clear.")

In [ ]:
# Interactive loop
def run_goal_agent():
    print("Goal-Based Agent (type 'exit' to quit)")
    print("Commands:")
    print("  recall <text>     - Find similar stored goals")
    print("  history           - View all stored goals")
    print("  clear             - Delete all stored goals")
    print("  <anything else>   - Save as new goal")

    while True:
        user_input = input("You > ").strip()

        if user_input.lower() == "exit":
            print("Exiting.")
            break

        elif user_input.lower() == "history":
            view_all_goals()

        elif user_input.lower() == "clear":
            clear_all_goals()

        elif user_input.lower().startswith("recall "):
            parts = user_input.split(maxsplit=1)
            if len(parts) == 2:
                recall_goals(parts[1])
            else:
                print("Usage: recall <keyword>")

        else:
            store_goal("user", user_input)

# Start agent
run_goal_agent()